In [34]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import random
import pandas as pd
from faker import Faker
import datetime

In [60]:
NUM_CLASSROOMS = 100
NUM_BUILDING_FLOORS = 4
NUM_CLASSROOMS_PER_FLOOR = 15
NUM_STUDENTS = 10
NUM_TEACHERS = 10

In [36]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
db = client['bda-project-oltp']

In [37]:
def insertData(collection_name, data):
    db = client['bda-project-oltp']
    collection = db[collection_name]
    collection.insert_many(data)

In [38]:
building_names = {
    'Ingeniería': [
        {'name': 'Aulas de ingeniería', '_id': ObjectId},
        {'name': 'CyT', '_id': ObjectId},
        {'name': 'Insignia', '_id': ObjectId},
    ],
    'Ciencias': [
        {'name': 'Aulas de ciencias', '_id': ObjectId},
        {'name': 'Posgrados de matemáticas y física', '_id': ObjectId},
        {'name': 'Yu Takeuchi', '_id': ObjectId}
    ],
    'Artes': [
        {'name': 'Aulas de artes', '_id': ObjectId},
        {'name': 'Conservatorio de música', '_id': ObjectId},
        {'name': 'Aulas de arquitectura', '_id': ObjectId}
    ]
}

classrooms_capacity = [20, 30, 40, 60, 80, 120, 200]
classroom_number = [floor * 100 + num for floor in range(1, NUM_BUILDING_FLOORS + 1) for num in range(1, NUM_CLASSROOMS_PER_FLOOR + 1)]
classroom_types = ['Salón', 'Laboratorio', 'Sala de computación', 'Auditorio']

auditory_number = ['AUD A', 'AUD B', 'AUD C', 'AUD D']

print(classroom_number)

[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415]


In [69]:
classrooms = []
for i in range(NUM_CLASSROOMS):
    capacity = random.choice(classrooms_capacity)
    name = random.choice(classroom_number)
    type = classroom_types[random.randint(0, 2)]
    has_pc = 1 if random.random() > 0.8 or type == "Sala de computación" else 0

    if(capacity > 80): 
        name = random.choice(auditory_number)
        type = classroom_types[3]
        has_pc = 0

    building_faculty = random.choice(list(building_names.keys()))
    aux_index = random.randint(0, 2)
    building_name = building_names[building_faculty][aux_index]['name']

    classrooms.append({
        'name': name,
        'capacity': capacity,
        'type': type,
        'has_pc': has_pc,
        'building': {
            'name': building_name,
            'latitude': 4.598990,
            'longitude': -74.072774
        }
    })

insertData('classrooms', classrooms)

In [53]:
path = '/home/camilo/Documents/BDA/Web Scraping/Subjects.csv'

subjects_df  = pd.read_csv(path)
subjects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sb_code          2010 non-null   object
 1   sb_name          2010 non-null   object
 2   sb_credits       2010 non-null   int64 
 3   sb_type          2010 non-null   object
 4   sb_campus_code   2010 non-null   int64 
 5   sb_campus_name   2010 non-null   object
 6   sb_faculty_code  2010 non-null   int64 
 7   sb_faculty_name  2010 non-null   object
 8   sb_is_active     2010 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 141.5+ KB


In [54]:
subjects_df = subjects_df.rename(columns={"sb_campus_code": "sb_program_code", "sb_campus_name": 'sb_program_name'})

In [55]:
subjects_df['sb_name'] = subjects_df['sb_name'].astype(str)
subjects_df['sb_credits'] = subjects_df['sb_credits'].astype(str)
subjects_df['sb_type'] = subjects_df['sb_type'].astype(str)
subjects_df['sb_program_name'] = subjects_df['sb_program_name'].astype(str)
subjects_df['sb_faculty_name'] = subjects_df['sb_faculty_name'].astype(str)

In [70]:
collection = client['bda-project-oltp']['subjects']
for i in subjects_df.index:
    
    result = collection.find_one({'name': subjects_df['sb_name'][i]})
    if not result:
        subject = {
            'name': subjects_df['sb_name'][i],
            'credits': str(subjects_df['sb_credits'][i]),
            'type': subjects_df['sb_type'][i]
        }
        collection.insert_one(subject)

In [71]:
programs_names = subjects_df['sb_program_name'].unique()
faculty_names = subjects_df['sb_faculty_name'].unique()

subject_collection = db['subjects']
programs = []

for i in range(len(programs_names)):
    subjects = []
    program_df = subjects_df[subjects_df['sb_program_name'] == programs_names[i]]
    for j in program_df.index:
        subject = subject_collection.find_one({'name': program_df['sb_name'][j]})
        subjects.append(subject['_id'])
        
    programs.append({
        'name': programs_names[i],
        'subjects': subjects
    })

insertData('programs', programs)


In [72]:
teacher = []
for i in range(NUM_TEACHERS):
    fake = Faker('es_CO')
    faculty = str.capitalize(random.choice(faculty_names).split()[2])
    teacher.append({
        'name': fake.name(),
        'email': fake.email(),
        'password': fake.sha256(raw_output=False),
        'salary': random.randint(5000000, 10000000),
        'faculty': faculty
    })

insertData('teachers', teacher)

In [23]:
def getSubjectFromProgram(program):
    program_collection = db['programs']
    program_subjects = program_collection.find_one({'name': program})['subjects']
    return random.choice(program_subjects)

In [62]:
print(getSubjectFromProgram('INGENIERÍA DE SISTEMAS Y COMPUTACIÓN'))

6489418b626faf7c48a6c397


In [73]:
campus_collection = db['campus']
classroom_collection = db['classrooms']
programs_collection = db['programs']

faculties = []
for i in range(len(faculty_names)):
    programs_names = subjects_df[subjects_df['sb_faculty_name'] == faculty_names[i]]['sb_program_name'].unique()
    programs_ids = []
    for j in range(len(programs_names)):
        program = programs_collection.find_one({'name': programs_names[j]})
        programs_ids.append(program['_id'])
    
    faculty_name = str.capitalize(faculty_names[i].split()[2])
    names = [i['name'] for i in list(building_names[faculty_name])]
    faculty_classrooms = classroom_collection.find({'building.name': {'$in': names}})

    faculties.append({
        'name': faculty_name,
        'programs': programs_ids,
        'classrooms': [classroom['_id'] for classroom in faculty_classrooms]
    })

campus = {
    'name': 'Campus Principal',
    'faculties': faculties
}

campus_collection.insert_one(campus)


In [64]:
def getSubjectDays() -> str:
    """
    Returns a string with the days of the week represented a binary string
    """

    # Containes the combinations of days in which a subject can be offered in format L-S
    dayCombinations = [
        '101000',
        '010100',
        '001010',
        '101010',
        '011100',
        '001110',
        '100000',
        '001000',
        '000100',
        '000010',
        '111110',
        '000001',
    ]
         
    return random.choice(dayCombinations)

In [74]:
teachers_collection = db['teachers']
classrooms_collection = db['classrooms']
subjects_collection = db['subjects']
campus_collection = db['campus']

faculty_name = ['Ingeniería', 'Ciencias', 'Artes']

hours = {
    "7:00": "9:00",
    "9:00": "11:00",
    "11:00": "13:00",
    "14:00": "16:00",
    "16:00": "18:00",
    "18:00": "20:00",
    "10:00": "13:00",
    "14:00": "17:00",
}

enrollments = []
for i in range(NUM_STUDENTS):
    fake = Faker('es_CO')
    falculty = random.choice(faculty_name)
    building = random.choice(building_names[faculty])['name']

    teachers_num = random.choices([1, 2, 3], [0.85, 0.15, 0.05])[0]
    result = teachers_collection.aggregate([
        {'$match': {'faculty': falculty}}, 
        {'$sample': {'size': teachers_num}},
        {'$project': {'_id': 1}}
    ])

    teachers = []
    aux = list(result)
    for i in range(len(aux)):
        teachers.append({
            '_id': aux[i]['_id'],
        })

    pipeline = [
        {'$match': {'faculties.name': falculty}},  # Filtrar por nombre de facultad
        {'$unwind': '$faculties'},  # Descomponer la lista de facultades en documentos individuales
        {'$match': {'faculties.name': faculty}},  # Filtrar por nombre de facultad
        {'$unwind': '$faculties.programs'},   # Descomponer la lista de programas en documentos individuales
        {'$sample': {'size': 1}}, # Obtener un programa aleatorio
        {'$project': {'_id': 0, 'programs': '$faculties.programs'}} # Proyectar solo el nombre del programa
    ]
    result = campus_collection.aggregate(pipeline)
    program = list(result)[0]['programs']
    subject = random.choice(programs_collection.find_one({'_id': program})['subjects'])

    classrooms_num = random.choices([1, 2, 3], [0.85, 0.15, 0.05])[0]
    classrooms_data = classrooms_collection.aggregate([
        {'$match': {'building.name': building}}, 
        {'$sample': {'size': classrooms_num}}
    ])
    
    classrooms = []
    start, end = random.choice(list(hours.items()))

    classrooms_data = list(classrooms_data)

    for i in range(len(classrooms_data)):
        classrooms.append({
            '_id': classrooms_data[i]['_id'],
            'days': getSubjectDays(),
            'shour': start,
            'fhour': end
        })

    #print([teacher['_id'] for teacher in teachers])
    date = fake.date_between(start_date='-10y', end_date='today')
    date = datetime.datetime.combine(date, datetime.time(0, 0))
    enrollments.append({
        'subject': subject,
        'date': date,
        'teachers': [ObjectId(teacher['_id']) for teacher in teachers],
        'classrooms': classrooms
    })

insertData('enrollments', enrollments)

In [129]:
insertData('enrollments', enrollments)

In [66]:
print(len(enrollments))

10


In [67]:
campus_collection = db['campus']
pipeline = [
    {'$match': {'faculties.name': 'Artes'}}, 
    {'$unwind': '$faculties'},  # Descomponer la lista de facultades en documentos individuales
    {'$match': {'faculties.name': 'Artes'}},  # Filtrar por nombre de facultad
    {'$unwind': '$faculties.programs'},   # Descomponer la lista de programas en documentos individuales
    {'$sample': {'size': 1}}, # Obtener un programa aleatorio
    {'$project': {'_id': 0, 'programs': '$faculties.programs'}} # Proyectar solo el nombre del programa
]

result = campus_collection.aggregate(pipeline)
program = list(result)[0]['programs']
print(program)

subject = random.choice(programs_collection.find_one({'_id': program})['subjects'])
print(subject)

648941e7626faf7c48a6c841
648941c4626faf7c48a6c4bb


In [37]:
teachers = teachers_collection.aggregate([
    {'$match': {'faculty': 'Artes'}}, 
    {'$sample': {'size': 1}}
])

print(list(teachers)[0]['_id'])

6473a37c35ee398eb5f01e9a


In [38]:
print(building_names['Artes'])

[{'name': 'Aulas de artes', '_id': <class 'bson.objectid.ObjectId'>}, {'name': 'Conservatorio de música', '_id': <class 'bson.objectid.ObjectId'>}, {'name': 'Aulas de arquitectura', '_id': <class 'bson.objectid.ObjectId'>}]


In [75]:
fake = Faker('es_CO')
programs_collection = db['programs']
enrollments_collection = db['enrollments']
students_info = []
programs_list = list(programs_collection.find())
enrollments_list = list(enrollments_collection.find())
for i in range(NUM_STUDENTS):

    students_info.append({
        'name': fake.name(),
        'email': fake.email(),
        'password': fake.sha256(raw_output=False),
        'programs': []
    })

for student in students_info:
    programs = [];
    num_program = random.choices([1, 2], [0.95, 0.05])[0]
    for j in range(num_program):
        program = random.choice(programs_list)

        num_semesters = random.randint(1, 15)

        programs.append({
            'id': program['_id'],
            'semesters': [],
        })

        initial_date = fake.date_between(start_date='-10y', end_date='-3y')
        initial_year = initial_date.year
        initial_semester = 1 if initial_date.month <= 6 else 2

        for i in range (num_semesters):
            subjects_num = random.choices([1, 2, 3, 4, 5, 6], [0.05, 0.05, 0.15, 0.35, 0.35, 0.05])[0]

            year = initial_year if i == 0 else initial_year + (i + 1)

            
            semester = (initial_semester % 2) if (initial_semester % 2) == 1 else (initial_semester % 2) + 2
            initial_semester += 1

            if(year == 2023 and semester == 2 or year > 2023 and semester <= 2): break
            print(year, semester)
            programs[j]['semesters'].append({
                'year': year,
                'semester': semester,
                'enrollments': []
            })
            enrollments = random.choices(enrollments_list, k=subjects_num)

            for enrollment in enrollments:
                programs[j]['semesters'][i]['enrollments'].append({
                    'enrollment': enrollment['_id'],
                    'grade': round(random.uniform(1, 5), 2)
                })
    
    student['programs'] = programs

insertData('students', students_info)


2017 2
2019 1
2020 2
2016 1
2015 1
2017 2
2018 1
2019 2
2020 1
2021 2
2022 1
2015 2
2017 1
2018 2
2019 1
2020 2
2021 1
2017 1
2019 2
2020 1
2021 2
2022 1
2018 1
2020 2
2021 1
2022 2
2023 1
2013 2
2015 1
2017 1
2019 2
2020 1
2021 2
2022 1
2018 2
2020 1
2021 2
2022 1
2015 1
2017 2
2018 1
2019 2
2020 1
2021 2
2022 1
